<a href="https://colab.research.google.com/github/KevinWang676/Bark-Voice-Cloning/blob/main/XTTS_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://huggingface.co/spaces/coqui/xtts

Cloning into 'xtts'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (372/372), done.
remote: Compressing objects: 100% (362/362), done.
remote: Total 376 (delta 209), reused 0 (delta 0), pack-reused 4
Receiving objects: 100% (376/376), 692.03 KiB | 3.76 MiB/s, done.
Resolving deltas: 100% (209/209), done.
Filtering content: 100% (2/2), 28.81 MiB | 5.39 MiB/s, done.


In [2]:
cd xtts

/content/xtts


In [3]:
pip install -r requirements.txt

Ignoring numpy: markers 'python_version > "3.10"' don't match your environment
Ignoring numba: markers 'python_version < "3.9"' don't match your environment
  Cloning https://github.com/coqui-ai/tts.git (to revision v0.19.1) to /tmp/pip-req-build-f5y_2n_u
  Running command git clone --filter=blob:none --quiet https://github.com/coqui-ai/tts.git /tmp/pip-req-build-f5y_2n_u
  Resolved https://github.com/coqui-ai/tts.git to commit 6fef4f9067c0647258e0cd1d2998716565f59330
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 89.4 MB/s eta

In [4]:
pip install gradio==3.42.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.3 MB/s eta 0:00:00
  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.3 MB/s eta 0:00:00
  

In [5]:
import gradio as gr

In [ ]:
import sys
import io, os, stat
import subprocess
import random
from zipfile import ZipFile
import uuid
import time
import torch
import torchaudio
# By using XTTS you agree to CPML license https://coqui.ai/cpml
os.environ["COQUI_TOS_AGREED"] = "1"

# langid is used to detect language for longer text
# Most users expect text to be their own language, there is checkbox to disable it
import langid
import base64
import csv
from io import StringIO
import datetime

import gradio as gr
from scipy.io.wavfile import write
from pydub import AudioSegment

from TTS.api import TTS
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.generic_utils import get_user_data_dir

HF_TOKEN = os.environ.get("HF_TOKEN")

from huggingface_hub import HfApi

# will use api to restart space on a unrecoverable error
api = HfApi(token=HF_TOKEN)
repo_id = "coqui/xtts"

# Use never ffmpeg binary for Ubuntu20 to use denoising for microphone input
print("Export newer ffmpeg binary for denoise filter")
ZipFile("ffmpeg.zip").extractall()
print("Make ffmpeg binary executable")
st = os.stat('ffmpeg')
os.chmod('ffmpeg', st.st_mode | stat.S_IEXEC)

# This will trigger downloading model
print("Downloading if not downloaded Coqui XTTS V1.1")
from TTS.utils.manage import ModelManager
model_name = "tts_models/multilingual/multi-dataset/xtts_v1.1"
ModelManager().download_model(model_name)
model_path = os.path.join(get_user_data_dir("tts"), model_name.replace("/", "--"))
print("XTTS downloaded")

config = XttsConfig()
config.load_json(os.path.join(model_path, "config.json"))

# it should be there just to be sure
if "ja" not in config.languages:
    config.languages.append("ja")

model = Xtts.init_from_config(config)
model.load_checkpoint(
    config,
    checkpoint_path=os.path.join(model_path, "model.pth"),
    vocab_path=os.path.join(model_path, "vocab.json"),
    eval=True,
    use_deepspeed=True
)
model.cuda()

# This is for debugging purposes only
DEVICE_ASSERT_DETECTED=0
DEVICE_ASSERT_PROMPT=None
DEVICE_ASSERT_LANG=None



#supported_languages=["en","es","fr","de","it","pt","pl","tr","ru","nl","cs","ar","zh-cn"]
supported_languages=config.languages

def predict(prompt, language, audio_file_pth, mic_file_path, use_mic, voice_cleanup, no_lang_auto_detect, agree,):
    if agree == True:


        if language not in supported_languages:
            gr.Warning(f"Language you put {language} in is not in is not in our Supported Languages, please choose from dropdown")

            return (
                    None,
                    None,
                    None,
                    None,
                )

        language_predicted=langid.classify(prompt)[0].strip() # strip need as there is space at end!

        # tts expects chinese as zh-cn
        if language_predicted == "zh":
            #we use zh-cn
            language_predicted = "zh-cn"

        print(f"Detected language:{language_predicted}, Chosen language:{language}")

        # After text character length 15 trigger language detection
        if len(prompt)>15:
            # allow any language for short text as some may be common
            # If user unchecks language autodetection it will not trigger
            # You may remove this completely for own use
            if language_predicted != language and not no_lang_auto_detect:
                #Please duplicate and remove this check if you really want this
                #Or auto-detector fails to identify language (which it can on pretty short text or mixed text)
                gr.Warning(f"It looks like your text isn’t the language you chose , if you’re sure the text is the same language you chose, please check disable language auto-detection checkbox" )

                return (
                        None,
                        None,
                        None,
                        None,
                    )


        if use_mic == True:
            if mic_file_path is not None:
               speaker_wav=mic_file_path
            else:
                gr.Warning("Please record your voice with Microphone, or uncheck Use Microphone to use reference audios")
                return (
                    None,
                    None,
                    None,
                    None,
                )

        else:
            speaker_wav=audio_file_pth


        # Filtering for microphone input, as it has BG noise, maybe silence in beginning and end
        # This is fast filtering not perfect

        # Apply all on demand
        lowpassfilter=denoise=trim=loudness=True

        if lowpassfilter:
            lowpass_highpass="lowpass=8000,highpass=75,"
        else:
            lowpass_highpass=""

        if trim:
            # better to remove silence in beginning and end for microphone
            trim_silence="areverse,silenceremove=start_periods=1:start_silence=0:start_threshold=0.02,areverse,silenceremove=start_periods=1:start_silence=0:start_threshold=0.02,"
        else:
            trim_silence=""

        if (voice_cleanup):
            try:
                out_filename = speaker_wav + str(uuid.uuid4()) + ".wav"  #ffmpeg to know output format

                #we will use newer ffmpeg as that has afftn denoise filter
                shell_command = f"./ffmpeg -y -i {speaker_wav} -af {lowpass_highpass}{trim_silence} {out_filename}".split(" ")

                command_result = subprocess.run([item for item in shell_command], capture_output=False,text=True, check=True)
                speaker_wav=out_filename
                print("Filtered microphone input")
            except subprocess.CalledProcessError:
                # There was an error - command exited with non-zero code
                print("Error: failed filtering, use original microphone input")
        else:
            speaker_wav=speaker_wav

        if len(prompt)<2:
            gr.Warning("Please give a longer prompt text")
            return (
                    None,
                    None,
                    None,
                    None,
                )
        if len(prompt)>1000:
            gr.Warning("Text length limited to 1000 characters for this demo, please try shorter text. You can clone this space and edit code for your own usage")
            return (
                    None,
                    None,
                    None,
                    None,
                )
        global DEVICE_ASSERT_DETECTED
        if DEVICE_ASSERT_DETECTED:
            global DEVICE_ASSERT_PROMPT
            global DEVICE_ASSERT_LANG
            #It will likely never come here as we restart space on first unrecoverable error now
            print(f"Unrecoverable exception caused by language:{DEVICE_ASSERT_LANG} prompt:{DEVICE_ASSERT_PROMPT}")

        try:
            metrics_text=""
            t_latent=time.time()

            # note diffusion_conditioning not used on hifigan (default mode), it will be empty but need to pass it to model.inference
            try:
                gpt_cond_latent, diffusion_conditioning, speaker_embedding = model.get_conditioning_latents(audio_path=speaker_wav)
            except Exception as e:
                print("Speaker encoding error", str(e))
                gr.Warning("It appears something wrong with reference, did you unmute your microphone?")
                return (
                    None,
                    None,
                    None,
                    None,
                )


            latent_calculation_time = time.time() - t_latent
            #metrics_text=f"Embedding calculation time: {latent_calculation_time:.2f} seconds\n"

            wav_chunks = []

            print("I: Generating new audio...")
            t0 = time.time()
            out = model.inference(
                prompt,
                language,
                gpt_cond_latent,
                speaker_embedding,
                diffusion_conditioning,
                decoder="ne_hifigan",
            )
            inference_time = time.time() - t0
            print(f"I: Time to generate audio: {round(inference_time*1000)} milliseconds")
            metrics_text+=f"Time to generate audio: {round(inference_time*1000)} milliseconds\n"
            real_time_factor= (time.time() - t0) / out['wav'].shape[-1] * 24000
            print(f"Real-time factor (RTF): {real_time_factor}")
            metrics_text+=f"Real-time factor (RTF): {real_time_factor:.2f}\n"
            torchaudio.save("output.wav", torch.tensor(out["wav"]).unsqueeze(0), 24000)
        except RuntimeError as e :
            if "device-side assert" in str(e):
                # cannot do anything on cuda device side error, need tor estart
                print(f"Exit due to: Unrecoverable exception caused by language:{language} prompt:{prompt}", flush=True)
                gr.Warning("Unhandled Exception encounter, please retry in a minute")
                print("Cuda device-assert Runtime encountered need restart")
                if not DEVICE_ASSERT_DETECTED:
                    DEVICE_ASSERT_DETECTED=1
                    DEVICE_ASSERT_PROMPT=prompt
                    DEVICE_ASSERT_LANG=language

                # just before restarting save what caused the issue so we can handle it in future
                # Uploading Error data only happens for unrecovarable error
                error_time = datetime.datetime.now().strftime('%d-%m-%Y-%H:%M:%S')
                error_data = [error_time, prompt, language, audio_file_pth, mic_file_path, use_mic, voice_cleanup, no_lang_auto_detect, agree]
                error_data = [str(e) if type(e)!=str else e for e in error_data]
                print(error_data)
                print(speaker_wav)
                write_io = StringIO()
                csv.writer(write_io).writerows([error_data])
                csv_upload= write_io.getvalue().encode()

                filename =  error_time+"_" + str(uuid.uuid4()) +".csv"
                print("Writing error csv")
                error_api = HfApi()
                error_api.upload_file(
                    path_or_fileobj=csv_upload,
                    path_in_repo=filename,
                    repo_id="coqui/xtts-flagged-dataset",
                    repo_type="dataset",
                )

                #speaker_wav
                print("Writing error reference audio")
                speaker_filename =  error_time+"_reference_"+ str(uuid.uuid4()) +".wav"
                error_api = HfApi()
                error_api.upload_file(
                    path_or_fileobj=speaker_wav,
                    path_in_repo=speaker_filename,
                    repo_id="coqui/xtts-flagged-dataset",
                    repo_type="dataset",
                )

                # HF Space specific.. This error is unrecoverable need to restart space
                api.restart_space(repo_id=repo_id)
            else:
                if "Failed to decode" in str(e):
                    print("Speaker encoding error", str(e))
                    gr.Warning("It appears something wrong with reference, did you unmute your microphone?")
                else:
                    print("RuntimeError: non device-side assert error:", str(e))
                    gr.Warning("Something unexpected happened please retry again.")
                return (
                        None,
                        None,
                        None,
                        None,
                    )
        return (
            gr.make_waveform(
                audio="output.wav",
            ),
            "output.wav",
            metrics_text,
            speaker_wav,
        )
    else:
        gr.Warning("Please accept the Terms & Condition!")
        return (
                None,
                None,
                None,
                None,
            )


title = "🐸🌊💕 XTTS — 只需3秒语音，一键复刻14种语言"

description = """
## <center>🌟 - 3秒语音开启真实拟声，支持中日英在内的14种语言！Powered by [Coqui AI](https://coqui.ai/) </center>
### <center>🌊 - 更多精彩应用，敬请关注[滔滔AI](http://www.talktalkai.com)；滔滔AI，为爱滔滔！💕</center>
"""

article = """
<p>Language Selectors:
Arabic: ar, Brazilian Portuguese: pt , Chinese: zh-cn, Czech: cs,<br/>
Dutch: nl, English: en, French: fr, Italian: it, Polish: pl,<br/>
Russian: ru, Spanish: es, Turkish: tr, Japanese: ja <br/>
</p>
### <center>注意❗：请不要生成会对个人以及组织造成侵害的内容，此程序仅供科研、学习及个人娱乐使用。</center>
<div class="footer">
            <p>🌊🏞️🎶 - 江水东流急，滔滔无尽声。 明·顾璘
            </p>
</div>
"""
examples = [
    [
        "Once when I was six years old I saw a magnificent picture",
        "en",
        "examples/female.wav",
        None,
        False,
        False,
        False,
        True,

    ],
    [
        "Lorsque j'avais six ans j'ai vu, une fois, une magnifique image",
        "fr",
        "examples/male.wav",
        None,
        False,
        False,
        False,
        True,
    ],
    [
        "Als ich sechs war, sah ich einmal ein wunderbares Bild",
        "de",
        "examples/female.wav",
        None,
        False,
        False,
        False,
        True,
    ],
    [
        "Cuando tenía seis años, vi una vez una imagen magnífica",
        "es",
        "examples/male.wav",
        None,
        False,
        False,
        False,
        True,
    ],
    [
        "Quando eu tinha seis anos eu vi, uma vez, uma imagem magnífica",
        "pt",
        "examples/female.wav",
        None,
        False,
        False,
        False,
        True,
    ],
    [
        "Kiedy miałem sześć lat, zobaczyłem pewnego razu wspaniały obrazek",
        "pl",
        "examples/male.wav",
        None,
        False,
        False,
        False,
        True,
    ],
    [
        "Un tempo lontano, quando avevo sei anni, vidi un magnifico disegno",
        "it",
        "examples/female.wav",
        None,
       False,
        False,
        False,
        True,
    ],
    [
        "Bir zamanlar, altı yaşındayken, muhteşem bir resim gördüm",
        "tr",
        "examples/female.wav",
        None,
        False,
        False,
        False,
        True,
    ],
    [
        "Когда мне было шесть лет, я увидел однажды удивительную картинку",
        "ru",
        "examples/female.wav",
        None,
       False,
        False,
        False,
        True,
    ],
    [
        "Toen ik een jaar of zes was, zag ik op een keer een prachtige plaat",
        "nl",
        "examples/male.wav",
        None,
       False,
        False,
        False,
        True,
    ],
    [
        "Když mi bylo šest let, viděl jsem jednou nádherný obrázek",
        "cs",
        "examples/female.wav",
        None,
       False,
        False,
        False,
        True,
    ],
    [
        "当我还只有六岁的时候， 看到了一副精彩的插画",
        "zh-cn",
        "examples/female.wav",
        None,
       False,
        False,
        False,
        True,
    ],
    [
        "かつて 六歳のとき、素晴らしい絵を見ました",
        "ja",
        "examples/female.wav",
        None,
        False,
        True,
        False,
        True,
    ],
]



gr.Interface(
    fn=predict,
    inputs=[
        gr.Textbox(
            label="请填写您想复刻的内容",
            placeholder="想说却还没说的 还很多 攒着是因为想写成歌",
        ),
        gr.Dropdown(
            label="请选择复刻内容对应的语言",
            choices=[
                "en",
                "es",
                "fr",
                "de",
                "it",
                "pt",
                "pl",
                "tr",
                "ru",
                "nl",
                "cs",
                "ar",
                "zh-cn",
                "ja"
            ],
            max_choices=1,
            value="zh-cn",
        ),
        gr.Audio(
            label="方法一：请上传您喜欢的语音文件",
            type="filepath",
            value="examples/female.wav",
        ),
        gr.Audio(source="microphone",
                 type="filepath",
                 label="方法二：请用麦克风上传您喜欢的声音"),
        gr.Checkbox(label="是否使用麦克风上传声音（默认为否）",
                    value=False,
                    ),
        gr.Checkbox(label="是否对上传的语音进行降噪处理（默认为否）",
                    value=False,
                    info="如果您上传的语音有明显噪声，请选择降噪",
                    ),
        gr.Checkbox(label="Do not use language auto-detect",
                    value=False,
                    visible=False,
                    info="Check to disable language auto-detection",),
        gr.Checkbox(
            label="Agree",
            value=True,
            visible=False,
            info="I agree to the terms of the Coqui Public Model License at https://coqui.ai/cpml",
        ),


    ],
    outputs=[
        gr.Video(label="专属语音的波形图"),
        gr.Audio(label="为您合成的专属语音",autoplay=True),
        gr.Text(label="Metrics", visible=False),
        gr.Audio(label="Reference Audio Used", visible=False),
    ],
    title=title,
    description=description,
    article=article,
    examples=examples,
).queue().launch(debug=True,show_api=True)

Export newer ffmpeg binary for denoise filter
Make ffmpeg binary executable
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v1.1
 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
XTTS downloaded
[2023-11-04 06:40:58,748] [INFO] [logging.py:93:log_dist] [Rank -1] DeepSpeed info: version=0.8.3, git-hash=unknown, git-branch=unknown
[2023-11-04 06:40:58,752] [WARNING] [config_utils.py:75:_process_deprecated_field] Config parameter replace_method is deprecated. This parameter is no longer needed, please remove from your call to DeepSpeed-inference
[2023-11-04 06:40:58,754] [WARNING] [config_utils.py:75:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2023-11-04 06:40:58,756] [INFO] [logging.py:93:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1
Installed CUDA version 11.8 does not match the version torch was compiled with 11.7 but 

Using /root/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu117/transformer_inference...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu117/transformer_inference/build.ninja...
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


Time to load transformer_inference op: 35.26722860336304 seconds
[2023-11-04 06:41:35,670] [INFO] [logging.py:93:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layers': -1, 'fp16': False, 'pre_layer_norm': True, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'q_int8': False, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 1, 'rotary_dim': -1, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': True, 'mlp_act_func_type': <ActivationFuncType.GELU: 1>, 'specialized_mode': False, 'training_mp_size': 1, 'bigscience_bloom': False, 'max_out_tokens': 1024, 'scale_attn_by_inverse_layer_idx': False, 'enable_qkv_quantization': False, 'use_mup': False, 'return_single_tuple': False}


Loading extension module transformer_inference...


Installed CUDA version 11.8 does not match the version torch was compiled with 11.7 but since the APIs are compatible, accepting this combination
Time to load transformer_inference op: 0.003851175308227539 seconds


...

/usr/local/lib/python3.10/dist-packages/matplotlib/axes/_axes.py:2229: RuntimeWarning: overflow encountered in int_scalars
  dx = [convert(x0 + ddx) - x for ddx in dx]
/usr/local/lib/python3.10/dist-packages/matplotlib/patches.py:739: RuntimeWarning: overflow encountered in int_scalars
  y1 = self.convert_yunits(self._y0 + self._height)


Detected language:en, Chosen language:en
I: Generating new audio...
I: Time to generate audio: 4109 milliseconds
Real-time factor (RTF): 0.2263596874566532
Detected language:en, Chosen language:en
I: Generating new audio...
I: Time to generate audio: 2806 milliseconds
Real-time factor (RTF): 0.22899703786516315
Detected language:en, Chosen language:en
I: Generating new audio...
I: Time to generate audio: 3593 milliseconds
Real-time factor (RTF): 0.23168362830823194
Detected language:en, Chosen language:en
Error: failed filtering, use original microphone input
I: Generating new audio...
I: Time to generate audio: 3597 milliseconds
Real-time factor (RTF): 0.23196151856707084
Detected language:en, Chosen language:en
Error: failed filtering, use original microphone input
I: Generating new audio...
I: Time to generate audio: 4133 milliseconds
Real-time factor (RTF): 0.22763659965277558
Detected language:en, Chosen language:en
Error: failed filtering, use original microphone input
I: Generat

/usr/local/lib/python3.10/dist-packages/matplotlib/axes/_axes.py:2229: RuntimeWarning: overflow encountered in int_scalars
  dx = [convert(x0 + ddx) - x for ddx in dx]
/usr/local/lib/python3.10/dist-packages/matplotlib/patches.py:739: RuntimeWarning: overflow encountered in int_scalars
  y1 = self.convert_yunits(self._y0 + self._height)


Detected language:en, Chosen language:en
I: Generating new audio...
I: Time to generate audio: 2370 milliseconds
Real-time factor (RTF): 0.23003857328284602


/usr/local/lib/python3.10/dist-packages/matplotlib/axes/_axes.py:2229: RuntimeWarning: overflow encountered in int_scalars
  dx = [convert(x0 + ddx) - x for ddx in dx]
/usr/local/lib/python3.10/dist-packages/matplotlib/patches.py:739: RuntimeWarning: overflow encountered in int_scalars
  y1 = self.convert_yunits(self._y0 + self._height)
